In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
TOPN_candidate = 100
SETS = [1]
VER = 6
SUBSETNUM = 3

TOPN_b2b = 20
TOPN_cartsOrders = 20
TOPN_clicks = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_click2order = 20

note = 'covisit_20_20_20_newSuggester2'
outputNote = 'covisit_20_20_20_newSuggester2_moreCovwgt'

predTypes = ['clicks', 'carts', 'orders']
type_labels = {'clicks':0, 'carts':1, 'orders':2}
type2covisitType = {'clicks': 'clicks', 'carts': 'cartsOrders', 'orders': 'cartsOrders'}

In [4]:
def load_df(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [5]:
for SET in SETS:
    coVisitSaveFolder = outputPath + f'coVisit_set{SET}_top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
    candidatesSavePath = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{note}/'
    candidatesSavePath_covwgt = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{outputNote}/'
    ! mkdir {candidatesSavePath_covwgt}
    if SET == 1:
        val_A = load_df('../input/split_2-1_pqt/test_parquets/*')
    elif SET == 2:
        val_A = pd.read_parquet('../input/splited/test.parquet')
    elif SET == 3:
        val_A = load_df('../input/parquets/test_parquets/*')
        # val_A = val_A.drop_duplicates((['session', 'aid']))
    for k in range(0, 8):
        if k == 0:
            covistWeight_clicks = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_clicks_v{VER}_{k}.pqt')
        else:
            covistWeight_clicks = pd.concat([covistWeight_clicks, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_clicks_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 8):
        if k == 0:
            covistWeight_cl2cl = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_c2c}_click2click_v{VER}_{k}.pqt')
        else:
            covistWeight_cl2cl = pd.concat([covistWeight_cl2cl, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_c2c}_click2click_v{VER}_{k}.pqt')], axis=0)

    for k in range(0, 8):
        if k == 0:
            covistWeight_cartsOders = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_cartsOrders_v{VER}_{k}.pqt')
        else:
            covistWeight_cartsOders = pd.concat([covistWeight_cartsOders, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_cartsOrders_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 2):
        if k == 0:
            covistWeight_b2b = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_b2b}_buy2buy_v{VER}_{k}.pqt')
        else:
            covistWeight_b2b = pd.concat([covistWeight_b2b, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_b2b}_buy2buy_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 8):
        if k == 0:
            covistWeightcl2ca = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2cart}_click2cart_v{VER}_{k}.pqt')
        else:
            covistWeightcl2ca = pd.concat([covistWeightcl2ca, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2cart}_click2cart_v{VER}_{k}.pqt')], axis=0)

    for k in range(0, 8):
        if k == 0: 
            covistWeightcl2or = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2order}_click2order_v{VER}_{k}.pqt')
        else:
            covistWeightcl2or = pd.concat([covistWeightcl2or, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2order}_click2order_v{VER}_{k}.pqt')], axis=0)


    for predType in predTypes:
        print('type:', predType)
        for s in range(SUBSETNUM):     
            print('sub:', s)    
            candidate = pd.read_parquet(candidatesSavePath + f'{predType}Candidate_{s}.pqt')
            candidate['session'] = candidate.session.astype('int32')
            df = val_A.merge(candidate[['session', 'aid']],on='session')
            if predType == 'clicks':
                df = df.merge(covistWeight_clicks, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_cl2cl, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df.loc[df.type.isin([1, 2]), 'wgt_y'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt', 'covWgt_cl2cl']
            elif predType == 'carts':
                df = df.merge(covistWeight_cartsOders, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_b2b, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeightcl2ca, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df.loc[df.type == 0, 'wgt_y'] = 0
                df.loc[df.type.isin([1, 2]), 'wgt'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean', 'wgt':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt', 'covWgt_b2b', 'covWgt_cl2cr']
            else:
                df = df.merge(covistWeight_cartsOders, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_b2b, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeightcl2or, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df.loc[df.type == 0, 'wgt_y'] = 0
                df.loc[df.type.isin([1, 2]), 'wgt'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean', 'wgt':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt', 'covWgt_b2b', 'covWgt_cl2or']
                
            candidate = candidate.merge(df, on=['session', 'aid'], how='left')
            candidate.to_parquet(candidatesSavePath_covwgt + f'{predType}Candidate_{s}.pqt')

mkdir: cannot create directory ‘../output/newSplited/candidates_set1_top100_covisit_20_20_20_newSuggester2_moreCovwgt/’: File exists
type: clicks
sub: 0
sub: 1
sub: 2
type: carts
sub: 0
sub: 1
sub: 2
type: orders
sub: 0
sub: 1
sub: 2
